In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deepsvg.svglib.svg import SVG

from deepsvg import utils
from deepsvg.difflib.tensor import SVGTensor
from deepsvg.svglib.utils import to_gif
from deepsvg.svglib.geom import Bbox
from deepsvg.svg_dataset import SVGDataset, load_dataset
from deepsvg.utils.utils import batchify, linear

import torch

import os
import ntpath
import re
from tqdm import tqdm
import pickle
import random

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from cairosvg import svg2png
from PIL import Image
import cv2

import pandas as pd
import numpy as np

from sklearn.manifold import TSNE

In [3]:
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu") 

In [12]:
dim_z = 128

data = 'serifs'
pretrained_folder = "pretrained/"+data
pretrained_file = data+"_unlabelled_{}.pth.tar".format(dim_z)
pretrained_path = os.path.join(pretrained_folder, pretrained_file)

from configs.deepsvg.serifs_unlabelled import Config

cfg = Config()
cfg.model_cfg.dim_z = dim_z
model = cfg.make_model().to(device)
utils.load_model(pretrained_path, model)
model.eval();

In [13]:
dataset = load_dataset(cfg)

In [14]:
def encode_icon(idx):
    data = dataset.get(id=idx, random_aug=False)
    model_args = batchify((data[key] for key in cfg.model_args), device)
    with torch.no_grad():
        z = model(*model_args, encode_mode=True)
    return z

In [15]:
# = z = encode_icon('10000_MyriadPro-LightSemiExtIt_fs_50_R_Layer 2')

In [16]:
# extract dataframe
df = dataset.df

# create variables to save image paths and features
tensors = np.zeros((len(df), dim_z))
png_paths = []
jpg_paths = []

# create folder to save images
dataset_folder = os.path.dirname(cfg.meta_filepath)
png_folder = os.path.join(dataset_folder, 'pngs')

# create corresponding jpg folder
jpg_folder = os.path.join(dataset_folder, 'jpgs')

# object saving pickle path
pickle_save_path = os.path.join('encoded_data', pretrained_file.split('.')[0]+'.pkl')

In [17]:
# generate tensors and images

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    # get id
    idx = row["id"]
    
    # encode and save tensor
    tensor = encode_icon(idx)
    tensors[index] = tensor.cpu().numpy().squeeze()
    
    # save png and jpg paths
    path = os.path.join(cfg.data_dir, idx+'.svg')
    png_path = os.path.join(png_folder, idx+'.png')
    png_paths.append(png_path)
    
    jpg_path = re.sub('png', 'jpg', png_path)
    jpg_paths.append(jpg_path)

100%|██████████| 24366/24366 [05:08<00:00, 78.99it/s]


In [18]:
# Save Objects

file = dict()
file['tensors'] = tensors
file['jpg_paths'] = jpg_paths
file['png_paths'] = png_paths

with open(pickle_save_path, 'wb') as handle:
    pickle.dump(file, handle)